In [1]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [4]:
# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cpu


In [5]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

Extracting ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw


Extracting ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw


Extracting ./mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw


Extracting ./mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw
Processing...



Done!


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [6]:
# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [8]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [11]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.314905
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.304803
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.299929
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.303702
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.304085
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.299433
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.294686
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.299772
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.304119
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.295014
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.295266
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.294970
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.289238
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.310683
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.299308
Train Epoch: 1 | Batch Status: 9600/60

Train Epoch: 2 | Batch Status: 19200/60000 (32%) | Loss: 0.833481
Train Epoch: 2 | Batch Status: 19840/60000 (33%) | Loss: 0.931903
Train Epoch: 2 | Batch Status: 20480/60000 (34%) | Loss: 0.652848
Train Epoch: 2 | Batch Status: 21120/60000 (35%) | Loss: 0.693065
Train Epoch: 2 | Batch Status: 21760/60000 (36%) | Loss: 0.503471
Train Epoch: 2 | Batch Status: 22400/60000 (37%) | Loss: 0.500127
Train Epoch: 2 | Batch Status: 23040/60000 (38%) | Loss: 0.790369
Train Epoch: 2 | Batch Status: 23680/60000 (39%) | Loss: 0.572955
Train Epoch: 2 | Batch Status: 24320/60000 (41%) | Loss: 0.599205
Train Epoch: 2 | Batch Status: 24960/60000 (42%) | Loss: 0.558704
Train Epoch: 2 | Batch Status: 25600/60000 (43%) | Loss: 0.358116
Train Epoch: 2 | Batch Status: 26240/60000 (44%) | Loss: 0.569574
Train Epoch: 2 | Batch Status: 26880/60000 (45%) | Loss: 0.515120
Train Epoch: 2 | Batch Status: 27520/60000 (46%) | Loss: 0.617122
Train Epoch: 2 | Batch Status: 28160/60000 (47%) | Loss: 0.498874
Train Epoc

Train Epoch: 3 | Batch Status: 38400/60000 (64%) | Loss: 0.224330
Train Epoch: 3 | Batch Status: 39040/60000 (65%) | Loss: 0.288003
Train Epoch: 3 | Batch Status: 39680/60000 (66%) | Loss: 0.272482
Train Epoch: 3 | Batch Status: 40320/60000 (67%) | Loss: 0.201435
Train Epoch: 3 | Batch Status: 40960/60000 (68%) | Loss: 0.291414
Train Epoch: 3 | Batch Status: 41600/60000 (69%) | Loss: 0.390284
Train Epoch: 3 | Batch Status: 42240/60000 (70%) | Loss: 0.490795
Train Epoch: 3 | Batch Status: 42880/60000 (71%) | Loss: 0.593586
Train Epoch: 3 | Batch Status: 43520/60000 (72%) | Loss: 0.661809
Train Epoch: 3 | Batch Status: 44160/60000 (74%) | Loss: 0.408813
Train Epoch: 3 | Batch Status: 44800/60000 (75%) | Loss: 0.256313
Train Epoch: 3 | Batch Status: 45440/60000 (76%) | Loss: 0.234684
Train Epoch: 3 | Batch Status: 46080/60000 (77%) | Loss: 0.277429
Train Epoch: 3 | Batch Status: 46720/60000 (78%) | Loss: 0.293398
Train Epoch: 3 | Batch Status: 47360/60000 (79%) | Loss: 0.550117
Train Epoc

Train Epoch: 4 | Batch Status: 56960/60000 (95%) | Loss: 0.243921
Train Epoch: 4 | Batch Status: 57600/60000 (96%) | Loss: 0.317173
Train Epoch: 4 | Batch Status: 58240/60000 (97%) | Loss: 0.176655
Train Epoch: 4 | Batch Status: 58880/60000 (98%) | Loss: 0.097888
Train Epoch: 4 | Batch Status: 59520/60000 (99%) | Loss: 0.120985
Training time: 0m 15s
Test set: Average loss: 0.0032, Accuracy: 9405/10000 (94%)
Testing time: 0m 17s
Train Epoch: 5 | Batch Status: 0/60000 (0%) | Loss: 0.216374
Train Epoch: 5 | Batch Status: 640/60000 (1%) | Loss: 0.115347
Train Epoch: 5 | Batch Status: 1280/60000 (2%) | Loss: 0.179836
Train Epoch: 5 | Batch Status: 1920/60000 (3%) | Loss: 0.296286
Train Epoch: 5 | Batch Status: 2560/60000 (4%) | Loss: 0.283465
Train Epoch: 5 | Batch Status: 3200/60000 (5%) | Loss: 0.368519
Train Epoch: 5 | Batch Status: 3840/60000 (6%) | Loss: 0.212291
Train Epoch: 5 | Batch Status: 4480/60000 (7%) | Loss: 0.330488
Train Epoch: 5 | Batch Status: 5120/60000 (9%) | Loss: 0.201

Train Epoch: 6 | Batch Status: 14720/60000 (25%) | Loss: 0.265685
Train Epoch: 6 | Batch Status: 15360/60000 (26%) | Loss: 0.183889
Train Epoch: 6 | Batch Status: 16000/60000 (27%) | Loss: 0.099359
Train Epoch: 6 | Batch Status: 16640/60000 (28%) | Loss: 0.060779
Train Epoch: 6 | Batch Status: 17280/60000 (29%) | Loss: 0.122155
Train Epoch: 6 | Batch Status: 17920/60000 (30%) | Loss: 0.210044
Train Epoch: 6 | Batch Status: 18560/60000 (31%) | Loss: 0.232605
Train Epoch: 6 | Batch Status: 19200/60000 (32%) | Loss: 0.316219
Train Epoch: 6 | Batch Status: 19840/60000 (33%) | Loss: 0.252011
Train Epoch: 6 | Batch Status: 20480/60000 (34%) | Loss: 0.130436
Train Epoch: 6 | Batch Status: 21120/60000 (35%) | Loss: 0.127635
Train Epoch: 6 | Batch Status: 21760/60000 (36%) | Loss: 0.242797
Train Epoch: 6 | Batch Status: 22400/60000 (37%) | Loss: 0.173274
Train Epoch: 6 | Batch Status: 23040/60000 (38%) | Loss: 0.180637
Train Epoch: 6 | Batch Status: 23680/60000 (39%) | Loss: 0.318968
Train Epoc

Train Epoch: 7 | Batch Status: 33280/60000 (55%) | Loss: 0.115928
Train Epoch: 7 | Batch Status: 33920/60000 (57%) | Loss: 0.079764
Train Epoch: 7 | Batch Status: 34560/60000 (58%) | Loss: 0.174864
Train Epoch: 7 | Batch Status: 35200/60000 (59%) | Loss: 0.202025
Train Epoch: 7 | Batch Status: 35840/60000 (60%) | Loss: 0.175278
Train Epoch: 7 | Batch Status: 36480/60000 (61%) | Loss: 0.063413
Train Epoch: 7 | Batch Status: 37120/60000 (62%) | Loss: 0.145922
Train Epoch: 7 | Batch Status: 37760/60000 (63%) | Loss: 0.113387
Train Epoch: 7 | Batch Status: 38400/60000 (64%) | Loss: 0.252735
Train Epoch: 7 | Batch Status: 39040/60000 (65%) | Loss: 0.086370
Train Epoch: 7 | Batch Status: 39680/60000 (66%) | Loss: 0.078000
Train Epoch: 7 | Batch Status: 40320/60000 (67%) | Loss: 0.283336
Train Epoch: 7 | Batch Status: 40960/60000 (68%) | Loss: 0.078681
Train Epoch: 7 | Batch Status: 41600/60000 (69%) | Loss: 0.050927
Train Epoch: 7 | Batch Status: 42240/60000 (70%) | Loss: 0.076594
Train Epoc

Train Epoch: 8 | Batch Status: 52480/60000 (87%) | Loss: 0.071286
Train Epoch: 8 | Batch Status: 53120/60000 (88%) | Loss: 0.033531
Train Epoch: 8 | Batch Status: 53760/60000 (90%) | Loss: 0.217912
Train Epoch: 8 | Batch Status: 54400/60000 (91%) | Loss: 0.040393
Train Epoch: 8 | Batch Status: 55040/60000 (92%) | Loss: 0.127989
Train Epoch: 8 | Batch Status: 55680/60000 (93%) | Loss: 0.238333
Train Epoch: 8 | Batch Status: 56320/60000 (94%) | Loss: 0.089359
Train Epoch: 8 | Batch Status: 56960/60000 (95%) | Loss: 0.060154
Train Epoch: 8 | Batch Status: 57600/60000 (96%) | Loss: 0.183111
Train Epoch: 8 | Batch Status: 58240/60000 (97%) | Loss: 0.298853
Train Epoch: 8 | Batch Status: 58880/60000 (98%) | Loss: 0.167370
Train Epoch: 8 | Batch Status: 59520/60000 (99%) | Loss: 0.198335
Training time: 0m 14s
Test set: Average loss: 0.0018, Accuracy: 9649/10000 (96%)
Testing time: 0m 15s
Train Epoch: 9 | Batch Status: 0/60000 (0%) | Loss: 0.142782
Train Epoch: 9 | Batch Status: 640/60000 (1%)